<a href="https://colab.research.google.com/github/meet-seth/rnd-digit-recognition/blob/master/DigitRecognition_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import random
import numpy as np
import torchvision
import torch.optim as optim
cuda0 = torch.device("cuda:0")
torch.cuda.set_device(cuda0)

In [ ]:
training_data = torchvision.datasets.MNIST(root = './data',train = True,download = True,transform = torchvision.transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(training_data,batch_size = 100,shuffle =True,pin_memory = True)
test_data = torchvision.datasets.MNIST(root = "./data",train = False,download = True,transform = torchvision.transforms.ToTensor())
testloader = torch.utils.data.DataLoader(test_data,batch_size = 100,shuffle = False,pin_memory = True)

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv_model = nn.Sequential( nn.Conv2d(1,32,3,stride=1,padding=1),
                              nn.ReLU(),
                              nn.MaxPool2d(2,stride=2),
                              nn.Conv2d(32,64,3,stride=1,padding=1),
                              nn.ReLU(),
                              nn.MaxPool2d(2,stride=2))
    self.linear_model = nn.Sequential(
                              nn.Linear(7*7*64,128),
                              nn.ReLU(),
                              nn.Dropout(p=0.5),
                              nn.Linear(128,10))
  def feedforward(self,x):
    x = self.conv_model(x)
    y = torch.flatten(x,1)
    return self.linear_model(y)

In [ ]:
model = Network()
model.cuda()

Network(
  (conv_model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_model): Sequential(
    (0): Linear(in_features=3136, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
def test_model():
  model.eval()
  correct = 0
  total = 0
  for data_test in testloader:
    image,label = data_test
    image,label = image.cuda(),label.cuda()
    output = model.feedforward(image)
    max_value,pred_class = torch.max(output,1)
    total += label.shape[0]
    correct += (pred_class == label).sum().item()
  accuracy = (100*correct)/total

  return (correct,total,accuracy)

def train_model(epochs): 
  model.train() 
  for i in range(epochs):
    for data_train in trainloader:
      image,label = data_train 
      image,label = image.cuda(),label.cuda() 
      optimizer.zero_grad()
      out = model.feedforward(image)
      ls = loss(out,label)
      ls.backward()
      optimizer.step()

      
    correct,total,accuracy = test_model()
    print("Epoch {0}: Correct: {1} Total: {2} Accuracy: {3}".format(i+1,correct,total,accuracy))

In [ ]:
#Training the model
epochs = 15
train_model(epochs)

Epoch 1: Correct: 9810 Total: 10000 Accuracy: 98.1
Epoch 2: Correct: 9857 Total: 10000 Accuracy: 98.57
Epoch 3: Correct: 9884 Total: 10000 Accuracy: 98.84
Epoch 4: Correct: 9909 Total: 10000 Accuracy: 99.09
Epoch 5: Correct: 9906 Total: 10000 Accuracy: 99.06
Epoch 6: Correct: 9908 Total: 10000 Accuracy: 99.08
Epoch 7: Correct: 9922 Total: 10000 Accuracy: 99.22
Epoch 8: Correct: 9913 Total: 10000 Accuracy: 99.13
Epoch 9: Correct: 9909 Total: 10000 Accuracy: 99.09
Epoch 10: Correct: 9901 Total: 10000 Accuracy: 99.01
Epoch 11: Correct: 9914 Total: 10000 Accuracy: 99.14
Epoch 12: Correct: 9907 Total: 10000 Accuracy: 99.07
Epoch 13: Correct: 9916 Total: 10000 Accuracy: 99.16
Epoch 14: Correct: 9894 Total: 10000 Accuracy: 98.94
Epoch 15: Correct: 9911 Total: 10000 Accuracy: 99.11


In [ ]:
model.to("cpu")
torch.save(model,"pytorchmodel.pth")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
